In [5]:
from get_test_input import read_input
import re
from itertools import product, combinations
from collections import defaultdict, deque
import math as m
import numpy as np
import heapq

### Day 1

In [ ]:
lines = read_input(day=1, round=1, test=False)
left_list, right_list = [int(l.split('   ')[0]) for l in lines], [int(l.split('   ')[1]) for l in lines]
left_list.sort()
right_list.sort()
print('Day 1 round 1 answer =', sum([abs(x-y) for x,y in zip(left_list, right_list)]))
print('Day 1 round 2 answer =', sum([x*right_list.count(x) for x in list(set(left_list).intersection(set(right_list))) ]))

### Day 2

In [ ]:
lines = read_input(day=2, round=1, test=False)
reports = [[int(x) for x in l.split(' ')] for l in lines]

def is_safe(report):
   return abs(sum([(report[i] - report[i+1])/abs(report[i] - report[i+1]) if 0 < abs(report[i] - report[i+1]) <= 3  else 0 for i in range(len(report)-1 )]))==len(report)-1

safe_reports = sum([is_safe(report) for report in reports])

def is_safe_with_damping(report):
    return is_safe(report) or any(is_safe(report[:i] + report[i + 1:]) for i in range(len(report)))

safe_reports_with_damping = sum([is_safe_with_damping(report) for report in reports])

print('Day 2 round 1 answer =', safe_reports)
print('Day 2 round 2 answer =', safe_reports_with_damping)

### Day 3

In [ ]:
multipliers = read_input(day=3, round=1, test=True,regex_screen=r'mul\(\d{1,3},\d{1,3}\)')

total_1 = sum([int(mul[mul.index('(')+1:mul.index(',')])*int(mul[mul.index(',')+1:-1]) for mul in multipliers])
multipliers_and_enablers = read_input(day=3, round=1, test=False,regex_screen=r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)')

skip = False
enabled_multipliers = list(filter(lambda x: x!='do()',[mul_enabler for mul_enabler in multipliers_and_enablers if  skip == False and mul_enabler != "do()" and  not (skip := mul_enabler == "don't()") or (mul_enabler == "do()" and not (skip := False))]))
total_2 = sum([int(mul[mul.index('(')+1:mul.index(',')])*int(mul[mul.index(',')+1:-1]) for mul in enabled_multipliers])
 
print('Day 3 round 1 answer =', total_1)
print('Day 3 round 2 answer =', total_2)

### Day 4

In [ ]:
grid = read_input(day=4, test=False)
def get_rows_columns_diagonals(grid):
    rows , columns = grid , [''.join(col) for col in zip(*grid)]
    num_rows, num_cols= len(columns), len(rows)

    diagonals = [''.join([grid[r + d][d] for d in range(min(num_rows - r, num_cols))]) for r in range(num_rows)] + [
                ''.join([grid[d][c + d] for d in range(min(num_rows, num_cols - c))]) for c in range(1, num_cols)] +[
                ''.join([grid[r + d][num_cols - 1 - d] for d in range(min(num_rows - r, num_cols))]) for r in range(num_rows)] + [
                ''.join([grid[d][c - d] for d in range(min(num_rows, c + 1))])for c in range(num_cols - 1)]
    return rows + columns + diagonals

def get_x_s(grid):
    return [(grid[r+1][c+1]+grid[r][c]+grid[r-1][c-1],grid[r-1][c+1]+grid[r][c]+grid[r+1][c-1]) for r in range(1, len(grid)-1) for c in range(1, len(grid[0])-1)]

def count_string_appearances(strings, substring):
    return sum([len(re.findall(f'(?={re.escape(substring)})', string))+len(re.findall(f'(?={re.escape(substring[::-1])})', string)) for string in strings])

def count_x_strings(xs, string):
    return sum([string_1 in (string ,string[::-1]) and string_2 in (string ,string[::-1]) for string_1, string_2 in xs])

round_1 = count_string_appearances(get_rows_columns_diagonals(grid), 'XMAS')
round_2 = count_x_strings(get_x_s(grid), 'MAS')

print('Day 4 round 1 answer =', round_1)
print('Day 4 round 2 answer =', round_2)

### Day 5

In [ ]:
#code to read input
rules, updates = read_input(day=5,test=False,input_delimter='\n\n')

rules = rules.split('\n')
updates = [update.split(',') for update in updates.split('\n')]

# Define rule dictionary - originally did a before and after dict, is that more efficient? don't think so

rule_dict = {}
for rule in rules:
    before , after = rule.split('|')
    if after in rule_dict:
        rule_dict[after].add(before)
    else:
        rule_dict[after] ={before}

# If return correction = True we are answering part two and we will fix the update and then return the new middle entry
def validate_update(rule_dict, original_update, return_correction = False):
    update = original_update.copy()
    i = 0
    while i < len(update):

        page , afters, should_be_before = update[i], set(update[i+1:]), {}
        
        if (page in rule_dict and (should_be_before := afters & rule_dict[page])):
            if not return_correction:
                return False
            else:
                update = update[:i] +[a for a in update[i+1:] if a in should_be_before ]+ [a for a in update[i:] if  a not in should_be_before ]
                i -= 1 
        i += 1
    
    else:
        if not return_correction:
            return  True
        elif update != original_update:
            return int(update[i//2])
        else:
            return 0

# Add up results for answers to part 1 and 2
round_1 = sum([int(update[len(update)//2 ]) for update in updates if validate_update(rule_dict,update)])
round_2 = sum([validate_update(rule_dict, update, return_correction=True) for update in updates])
print('Day 5 round 1 answer =', round_1)
print('Day 5 round 2 answer =', round_2)


### Day 6   

In [ ]:
positions = read_input(day=6,test=False,input_delimter='\n')
max_row, max_col = len(positions), len(positions[0])
position = ''.join(positions).index('^')//max_col, ''.join(positions).index('^') % max_col

obstacles =  set([(i, j ) for i , position in enumerate(positions) for j in range(len(position)) if  positions[i][j] == '#'])

def create_trail(position, obstacles, max_row, max_col):
    trail = [position]
    y_dir , x_dir, rotations = 2, 1 , [-1,0,1,0]
    rotation_trail = {(position,(y_dir,x_dir))}
    while 0< position[0] < max_row-1 and 0<position[1] < max_col -1:
        new_position =  position[0]-rotations[y_dir] , position[1] + rotations[x_dir]
        if new_position in obstacles:
            y_dir, x_dir = (y_dir + 1) % 4, (x_dir + 1) % 4
        elif  (new_position,(y_dir, x_dir)) in  rotation_trail:
            return trail, rotation_trail, True
        else:
            position = new_position
            trail.append(position)
            rotation_trail.add((position,(y_dir, x_dir)))
    return trail, rotation_trail, False

def find_loops(trail, obstacles, max_row, max_col):
    new_obstacles = set()
    for i, position in enumerate(trail[1:]):
        if position not in new_obstacles and position not in trail[:i]:
            _, _, loop_found = create_trail(trail[0],obstacles|{position},max_row, max_col)
            if loop_found:
                new_obstacles.add(position)
    return new_obstacles
round_1_trail, round_1_rotation_trail, loop_found = create_trail(position, obstacles, max_row, max_col)
print('Day 6 round 1 answer =', len(set(round_1_trail)))
new_obstacles = find_loops(round_1_trail, obstacles, max_row, max_col)
print('Day 6 round 2 answer =', len(new_obstacles))


### Day 7

In [ ]:
eqn_dict = [(int(line.split(': ')[0]),[int(x) for x in line.split(': ')[1].split(' ')]) for line in read_input(day=7,test=False)]

def validate_eqns(numbers,value,op_options):
    if not all(op_option in {'*','+','||'} for op_option in op_options):
        raise ValueError(f"Invalid input: {op_options}. All items must be in '*','+','||'.")    
    num_ops = len(numbers) - 1
    perms = product(op_options, repeat=num_ops)
    for ops in perms:
        current_result = numbers[0]
        for i, op in enumerate(ops):
            if op == '+':
                current_result = current_result + numbers[i + 1]
            elif  op == '*':
                current_result = current_result * numbers[i + 1]
            else:
                current_result = int(f'{current_result}{numbers[i+1]}')
        if current_result == value:
            return value
    return 0

round_1 = sum([validate_eqns(numbers, value, ['+', '*']) for value, numbers in eqn_dict])
print('Day 7 round 1 answer =',round_1)

round_2 = sum([validate_eqns(numbers, value, ['+', '*','||']) for value, numbers in eqn_dict])
print('Day 7 round 2 answer =',round_2)



### Day 8

In [ ]:
node_dict , lines = defaultdict(list), read_input(day=8,test=False)
for key, value in [(char,(i,j))  for i, line in enumerate(lines) for j, char in enumerate(line) if char != '.']:
    node_dict[key].append(value)
max_x, max_y = len(lines) - 1, len(lines[0]) - 1
antinodes, meta_nodes = [], []
for char , coords in node_dict.items():
    coord_pairs = combinations(coords, 2)
    for coord_1, coord_2  in coord_pairs:
        x_1, y_1, x_2, y_2 = *coord_1, *coord_2
        dx , dy = x_2 - x_1 , y_2 - y_1
        divisor = m.gcd(dx, dy)
        unit_dx, unit_dy = dx // divisor, dy // divisor
        for direction in [-1,1]:
            new_x_1 , new_x_2, new_y_1, new_y_2 = x_1, x_2, y_1, y_2
            while 0 <= new_x_1 <= max_x and 0<= new_y_1 <= max_y :
                meta_nodes.append((new_x_1, new_y_1))
                if (new_x_1 == x_1 - dx and new_y_1 == y_1 - dy) or (new_x_1 == x_2 + dx and new_y_1 == y_2 + dy):
                    antinodes.append((new_x_1, new_y_1))
                new_x_1 , new_y_1 = new_x_1 + direction * unit_dx , new_y_1 + direction * unit_dy
print('Day 8 round 1 answer =',len(set(antinodes)))
print('Day 8 round 2 answer =',len(set(meta_nodes)))


### Day 9

In [ ]:
disk_info = [[int(str(i//2))]*int(num)  if i % 2 == 0 else [-1]*int(num)  for i , num in enumerate(read_input(day=9,test=False)[0])]
disk = [file for files in disk_info for file in files ]

new_disk_1 , new_disk_2 = disk.copy(), disk.copy()

# Part 1
space_locations , number_locations = [i for i, x in enumerate(disk) if x == -1], [i for i, x in enumerate(disk) if x != -1]

for i, space_location in enumerate(space_locations):
    num_location = number_locations[-(i+1)]
    if num_location < space_location:
        break
    else:
        new_disk_1[space_location] , new_disk_1[num_location] = new_disk_1[num_location], -1


# Part 2
occupied_blocks, empty_blocks = [], []
start_of_block = 0
for i in range(1, len(disk)):
    if disk[i] != disk[start_of_block]:
        if disk[start_of_block] == -1:
            empty_blocks.append([start_of_block, i - 1])
        else:
            occupied_blocks.append([start_of_block, i - 1])
        start_of_block = i
if disk[start_of_block] != -1:
    occupied_blocks.append([start_of_block, len(disk) - 1])


for block_start, block_end in occupied_blocks[::-1]:
    block_size = block_end - block_start + 1
    for empty_block in empty_blocks:
        empty_start, empty_end = empty_block
        empty_size = empty_end - empty_start + 1
        if empty_start > block_start:
            break
        if empty_size >= block_size:
            new_disk_2[empty_start : empty_start + block_size] = disk[
                block_start : block_start + block_size
            ]
            new_disk_2[block_start : block_start + block_size] = [-1] * block_size
            empty_block[0] += block_size
            if empty_block[0] > empty_block[1]:
                empty_blocks.remove(empty_block)
            break
print('Day 9 round 1 answer =',sum([i*x for i, x in enumerate(new_disk_1) if x !=-1]))
print('Day 9 round 2 answer =',sum([i*x for i, x in enumerate(new_disk_2) if x !=-1]))

### Day 10  

In [ ]:
height_dict = defaultdict(set)
map = read_input(day=10,test=False)
for i , line in enumerate(map):
    for j , height in enumerate(line):
        height_dict[int(height)].add((i,j))

def find_neighbours(coordinates, max_x, max_y):
     return {(new_x, new_y) for x, y in coordinates for new_x, new_y in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
              if 0 <= new_x <= max_x and 0 <= new_y <= max_y
    }

round_1, round_2 = 0, 0
max_x , max_y  = len(map), len(map[0])
for start_position in height_dict[0]:
    coords = {start_position}
    trails = defaultdict(int)
    trails[start_position] = 1
    for i in range(1,10):
        #Part 1
        coords = find_neighbours(coords, max_x, max_y).intersection(height_dict[i])
        #Part 2
        next_trails = defaultdict(int) 
        for coord, trail_count in trails.items():
            neighbours = find_neighbours({coord}, max_x, max_y)
            valid_neighbours = neighbours.intersection(height_dict[i])
            for neighbour in valid_neighbours:
                 next_trails[neighbour] += trail_count
        trails = next_trails 
    round_1 += len(coords)
    round_2 += sum(trails.values())
print('Day 10 round 1 answer =', round_1)
print('Day 10 round 2 answer =', round_2)


### Day 11

In [ ]:
power_of_10 = {i: 10 ** i for i in range(20)} 

stone_dict = defaultdict(int)
for stone in map(int, read_input(day=11, test=False, input_delimter=' ')):
    stone_dict[stone] += 1


def blink(stone):
    if stone == 0:
        return [1]
    else:
        num_digits = int(m.log10(stone)) + 1  
        if num_digits % 2 == 0:
            divisor = power_of_10[num_digits // 2]  
            return [stone // divisor, stone % divisor]
        else:
            return [stone * 2024]

# Main loop
new_stone_dict = defaultdict(int)
for i in range(75):
    for stone, count in stone_dict.items():
        new_stones = blink(stone)
        for new_stone in new_stones:
            new_stone_dict[new_stone] += count
    stone_dict, new_stone_dict = new_stone_dict, defaultdict(int)  
    if i == 24:
        round_1 = sum(stone_dict.values()) 

round_2 = sum(stone_dict.values()) 
print('Day 11 round 1 answer =', round_1)
print('Day 11 round 2 answer =', round_2)

### Day 12

In [ ]:
garden = read_input(day=12, test=False)

def find_regions(coordinates):
    directions = [1,-1,1j,-1j] 
    visited , regions = set(), []

    def search_coords(coord, group):
        if coord in visited:
            return
        visited.add(coord)
        group.append(coord)
        for d in directions:
            neighbour = coord + d
            if neighbour in coordinates and neighbour not in visited:
                search_coords(neighbour, group)

    for coord in coordinates:
        if coord not in visited:
            group = []
            search_coords(coord, group)
            regions.append(group)

    return regions


def calc_area(region):
    return len(region)

def calc_perimeter(region):
    return len( [coord + d for coord in region for d in [1,-1,1j, -1j]  if coord +d not in region])

def calc_sides(region):
    perimiter_coords =  {(coord,d) for d in (+1,-1,+1j,-1j) for coord in region if coord+d not in region}
    roots_of_edges =  perimiter_coords - {(coord+d*1j, d) for coord, d in perimiter_coords}
    return len(roots_of_edges)

def calc_price(region):
    return calc_area(region) * calc_perimeter(region)

def calc_discount_price(region):
    return calc_area(region) * calc_sides(region)


max_x , max_y = len(garden), len(garden[0])

letter_locations , regions = defaultdict(set), defaultdict(set)
for x , line in enumerate(garden):
    for y, letter in enumerate(line):
            letter_locations[letter].add(x+y*1j)

for letter, coordinates in letter_locations.items():
    regions[letter] = find_regions(coordinates)

round_1 = sum([calc_price(plot) for letter, region in regions.items()  for plot in region])
round_2 = sum([calc_discount_price(plot) for letter, region in regions.items()  for plot in region])
print('Day 12 round 1 answer =', round_1)
print('Day 12 round 2 answer =', round_2)



### Day 13

In [ ]:
challenges = [challenge.split('\n') for challenge in read_input(day=13, test=False,input_delimter='\n\n')]

def parse_to_arrays(input_strings):

    x_values = []
    y_values = []
    prize_values = []
    for string in input_strings:
        if string.startswith("Button"):
            parts = string.split(", ")
            x_values.append(int(parts[0].split("+")[1]))
            y_values.append(int(parts[1].split("+")[1]))
        elif string.startswith("Prize"):
            parts = string.split(", ")
            prize_values.append(int(parts[0].split("=")[1]))
            prize_values.append(int(parts[1].split("=")[1]))

    buttons_array = np.array([x_values, y_values])
    prizes_array = np.array([prize_values]).T  
    return buttons_array, prizes_array

round_1, round_2 = 0, 0
for challenge in challenges:
    button_array , prize_array = parse_to_arrays(challenge)
    second_column = prize_array + 10000000000000 
    full_prize_array = np.hstack((prize_array, second_column))
    try:
        inverse =  np.linalg.inv(button_array)
        solution =np.round( inverse @ full_prize_array, decimals=2) # I ACKNOWLEDGE THIS IS A COMPLETE FLUKE BUT I WAS NOT EXACTLY FEELING GREAT
        if solution[0][0].is_integer() and solution[1][0].is_integer():
            round_1 += 3*solution[0][0]+solution[1][0]
        if solution[0][1].is_integer() and solution[1][1].is_integer():
            round_2 += 3*solution[0][1]+solution[1][1]
    except np.linalg.LinAlgError:
        pass
print('Day 13 round 1 answer =', int(round_1))
print('Day 13 round 2 answer =', int(round_2))

### Day 14

In [ ]:
test = False
positions_velocities = [(int(y[y.index('=')+1:].split(',')[0]), int(y.split(',')[1]) ) for x in read_input(day=14, test=test) for y in x.split(' ')]
positions, velocities = [x for i, x in enumerate(positions_velocities) if i %2 ==0], [x for i, x in enumerate(positions_velocities ) if i %2 ==1]
if test:
    x_midline , x_max , y_midline , y_max = 5, 11, 3, 7
else:
    x_midline , x_max , y_midline , y_max = 50,101, 51, 103

def quadrant_count(positions, x_midline, y_midline):
    quadrants = {'top-left': 0, 'top-right': 0, 'bottom-left': 0, 'bottom-right': 0}
    for x, y in positions:
        if x == x_midline or y == y_midline:
            continue
        if x < x_midline and y < y_midline:
            quadrants['bottom-left'] += 1
        elif x < x_midline and y > y_midline:
            quadrants['top-left'] += 1
        elif x > x_midline and y < y_midline:
            quadrants['bottom-right'] += 1
        elif x > x_midline and y > y_midline:
            quadrants['top-right'] += 1
    return quadrants

# Round 1

final_positions = []
for p ,v in zip(positions, velocities):
    new_x, new_y = (p[0]+ 100*v[0]) % x_max,( p[1]+100*v[1] )% y_max
    final_positions.append((new_x, new_y))

round_1 = 1
for x in quadrant_count(final_positions, x_midline, y_midline).values():
    round_1 *= x


# Round 2 - i had no idea what the tree would look like to i guessed it would have unique positions
round_2 = 0
while True:
    unique_positions = set(positions)
    if len(unique_positions) == len(positions):
        break
    for i in range(len(positions)):
        x, y, v_x, v_y = *positions[i], *velocities[i]
        new_x, new_y = (x+ v_x) % x_max,( y+v_y )% y_max
        positions[i] = (new_x, new_y)
    round_2 += 1
    if T > 10_000:
        break
print('Day 14 round 1 answer =', round_1)
print('Day 14 round 2 answer =', round_2)


### Day 15

In [ ]:
warehouse, moves = read_input(day=15,test=False, input_delimter='\n\n')
moves = moves.replace('\n','')
positions_dict, wide_positions_dict = defaultdict(list), defaultdict(list)
for x, line in enumerate(warehouse.split('\n')):
    for y, letter in enumerate(line):
        positions_dict[letter].append(x+y*1j)
        if letter == '@':
            wide_positions_dict[letter].append(x+2*y*1j)
        elif letter == '#':
            wide_positions_dict[letter] += [x+2*y*1j, x+(2*y+1)*1j]
        else:
            wide_positions_dict[letter].append((x+2*y*1j, x+(2*y+1)*1j))

        

def make_move(direction, position):
    move_dict = {'^': -1,  'v': 1, '>': 1j,  '<': -1j }
    return position + move_dict[direction]
    
def gps_coord(position):
    return 100*position.real + position.imag

for direction in moves:
    
    #Round 1
    new_robot_position = make_move(direction, positions_dict['@'][0])
    new_box_positions = positions_dict['O'][:]
    if new_robot_position in positions_dict['#']:
        change_position = False
    elif move_boxes := new_robot_position in positions_dict['O']:
        box_to_move = new_box_positions.pop(new_box_positions.index(new_robot_position))
        while move_boxes == True:
            new_box_position = make_move(direction, box_to_move)
            if new_box_position in positions_dict['#']:
                move_boxes = False
                change_position = False
            elif new_box_position in positions_dict['O']:
                new_box_positions.append(new_box_position)
                box_to_move = new_box_positions.pop(new_box_positions.index(new_box_position))
            else:
                change_position = True
                new_box_positions.append(new_box_position)
                move_boxes = False
    else:
        change_position = True
    if change_position:
        positions_dict['@'] = [new_robot_position]
        positions_dict['O'] = new_box_positions

    #Round 2
    new_wide_robot_position = make_move(direction, wide_positions_dict['@'][0])
    new_wide_box_positions = wide_positions_dict['O'][:]

    if new_wide_robot_position in wide_positions_dict['#']:
        change_wide_position = False

    elif move_wide_boxes := any([new_wide_robot_position in x for x in wide_positions_dict['O']]):

        index_to_pop = [i for i, x in enumerate(new_wide_box_positions) if new_wide_robot_position in x ][0]
        wide_box_to_move = new_wide_box_positions.pop(index_to_pop)
        wide_boxes_to_move = {wide_box_to_move}

        while move_wide_boxes == True:
            new_wide_boxes_to_move = set()
            new_wide_box_positions_to_add = set()
            for wide_box_to_move in wide_boxes_to_move:
                new_wide_box_position = (make_move(direction, wide_box_to_move[0]), make_move(direction, wide_box_to_move[1]))
                
                if new_wide_box_position[0] in wide_positions_dict['#'] or new_wide_box_position[1] in wide_positions_dict['#']:
                    move_wide_boxes = False
                    change_wide_position = False
                    break

                elif any([new_wide_box_position[0] in x for x in new_wide_box_positions]) or any([new_wide_box_position[1] in x for x in new_wide_box_positions]):
                    indexes_to_pop = sorted([i for i, x in enumerate(new_wide_box_positions) if new_wide_box_position[0] in x or  new_wide_box_position[1] in x], reverse=True)
                    for i in indexes_to_pop:
                        new_wide_box_to_move = new_wide_box_positions.pop(i)
                        new_wide_boxes_to_move.add(new_wide_box_to_move)
                    new_wide_box_positions_to_add.add(new_wide_box_position)

                else:
                    change_wide_position = True
                    new_wide_box_positions_to_add.add(new_wide_box_position)

            wide_boxes_to_move =  new_wide_boxes_to_move.copy()
            new_wide_box_positions += list(new_wide_box_positions_to_add)
            move_wide_boxes = len(wide_boxes_to_move) > 0 and move_wide_boxes
       
    else:
        change_wide_position = True

    if change_position:
        positions_dict['@'] = [new_robot_position]
        positions_dict['O'] = new_box_positions

    if change_wide_position:
        wide_positions_dict['@'] = [new_wide_robot_position]
        wide_positions_dict['O'] = new_wide_box_positions
    
print('Day 15 round 1 answer =', int(sum([gps_coord(position) for position in positions_dict['O']])))
print('Day 15 round 2 answer =', int(sum([gps_coord(position[0]) for position in wide_positions_dict['O']])))




### Day 16

In [ ]:
# I am sick of not learning the actual comsci algos needed for these so decided to do some research before starting, 

# Initially for my round 1 answer i used an A* algo, which was absolutely rapid for part 1 but by definition was against the point for part two
# This algorithm prioritises guesses by a best guess which in my case was minimising the manhatten distance
# This is much more efficient than dijkstras as you don't compute as many paths
# This is all gone now! Part two ruined it. Since i had refused dijkstra's once i tried desparately to adapt my code, but really it's all the same thing

# One thing I did which was sensible is only run it on coords with no obstacles in the first place, this simplifies loads of mess

all_objects = [(object, i, j) for i , line in enumerate( read_input(day=16, test=False)) for j, object in enumerate(line) ]
maze = {(i,j) for object, i , j in all_objects if object != '#'}
start , end = [(i,j) for object, i , j in all_objects if object == 'S'][0] ,[(i,j) for object, i , j in all_objects if object == 'E'][0]

def both_rounds_in_one(maze, start, end):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  
    best = float('inf')
    initial_direction = 0 
    visited_points = []


    # Priority queue: (priority, cost, position, direction, path) - this decides what node will be checked first
    # A very fun note:
        # If you include the equivalent of an identity column in sql, like an sql index it is faster to sort
        # I found this idea whilst researching solutions
        # I have left it out but may put it in a note
        
    priority_queue = [(0, start, initial_direction, [start])]
    costs = defaultdict(lambda: float('inf')) #if it's not in the dict of costs, make it a max value

    while priority_queue:

        cost, current, direction, path = heapq.heappop(priority_queue)
        
        if cost > costs[current, direction]:
            continue
        else:
            costs[(current, direction)] = cost
        # End reached
        if current == end and cost <= best:
            best = cost
            visited_points += path 
            

        for new_direction, (dx, dy) in enumerate(directions):
            next_coord = (current[0] + dx, current[1] + dy)
            if next_coord in maze: 

                rotation_cost = 1000 if direction != new_direction else 0

                new_cost = cost+ 1 + rotation_cost
               
                heapq.heappush(priority_queue, (new_cost, next_coord, new_direction, path + [next_coord]))

    return best, set(visited_points)


cost, visited_points = both_rounds_in_one(maze, start, end)

print('Day 16 round 1 answer =', cost)
print('Day 16 round 2 answer =', len(visited_points))


### Day 17

In [222]:
register_raw, program_raw  = read_input(day=17, test=False, input_delimter='\n\n')
register_dict = {reg.split(': ')[0][-1]:int(reg.split(': ')[1]) for reg in register_raw.split('\n')}
program = list(map(int, program_raw.split(': ')[1].split(',')))

def run_program(program, register_dict):
    output_list, instruction_pointer = [], 0
    def combo(operand):
        if operand <= 3: 
            return operand
        elif operand == 4:
            return register_dict['A']
        elif operand == 5:
            return register_dict['B']
        elif operand == 6:
            return register_dict['C']
        else:
            print('7 IS RESERVED! BUT WE GOT A 7!')

    def adv(operand):
        register_dict['A'] = register_dict['A']//2**combo(operand)

    def bxl(operand):
        register_dict['B'] = register_dict['B'] ^ operand

    def bst(operand):
        register_dict['B'] = combo(operand) % 8

    def jnz(operand):
        nonlocal instruction_pointer
        if register_dict['A'] != 0:
            instruction_pointer = operand

    def bxc(operand):
        register_dict['B'] = register_dict['B'] ^ register_dict['C']

    def out(operand):
        output_list.append(str(combo(operand)%8))

    def bdv(operand):
        register_dict['B'] = register_dict['A']//2**combo(operand)

    def cdv(operand):
        register_dict['C'] = register_dict['A']//2**combo(operand)

    while instruction_pointer < len(program):

        opcode, operand , current_instruction_pointer = program[instruction_pointer], program[instruction_pointer+1], instruction_pointer

        match opcode:
            case 0:
                adv(operand)
            case 1:
                bxl(operand)
            case 2:
                bst(operand)
            case 3:
                jnz(operand)
            case 4:
                bxc(operand)
            case 5:
                out(operand)
            case 6:
                bdv(operand)
            case 7:
                cdv(operand)

        if instruction_pointer == current_instruction_pointer:
            instruction_pointer +=2
        

    return ','.join(output_list)

print('Day 17 round 1 answer =', run_program(program, register_dict))
# The key insight is that the output is writing the input in base 8 sort of. 
# Therefore we need to find a number mod 8 that gives us the last digit of the program
# We would then shift the base 8 representation of this number one base 8 unit along (equivalent to adding a zero)
# Therefore multiply by 8 (equivalent to multiply 10) and repeat to find the next
# we will need to multiply by 8 enough times to have the length of the program
# Also only need to worry about register A as it overwrites everything else
# If something fails you need to go back down levels to find the next candidate
# this is recursion

target = program[::-1]
def reverse_program(program, a=0, depth=0):
    if depth == len(program):
        return a
    for i in range(8):
        register_dict['A'] = a*8 + i
        output = list(map(int,run_program(program, register_dict).split(',')))
        if output[0] == program[::-1][depth]:
            if result := reverse_program(program,(a*8 + i), depth+1): 
                return result
    return 0

print('Day 17 round 2 answer =', reverse_program(program))

Day 17 round 1 answer = 2,3,6,2,1,6,1,2,1
Day 17 round 2 answer = 90938893795561


### Day 18

In [291]:
test = False
obstacles = [(int(x.split(',')[0]),int(x.split(',')[1])) for x in read_input(day=18, test=test)]
grid = [(i,j) for i in range(71) for j in range(71)] if not test else [(i,j) for i in range(7) for j in range(7)] 

start_moving = 1024 if not test else 12

maze = {xy for xy in grid if xy not in obstacles[:1024]} if not test else{xy for xy in grid if xy not in obstacles[:12]}
start = (0,0)
end = (70,70) if not test else (6,6)

def find_route(maze, start, end):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Movement directions

    def best_guess(current, goal):
        dx, dy = abs(current[0] - goal[0]), abs(current[1] - goal[1])
        return dx + dy

    priority_queue = [(0 + best_guess(start, end), 0, start)]  # (total_cost, length, current)
    lengths = {start: 0}

    while priority_queue:
        _,length, current = heapq.heappop(priority_queue)

        if current == end:
            return length  

        for dx, dy in directions:
            next_coord = (current[0] + dx, current[1] + dy)
            if next_coord in maze:
                new_length = length + 1

                if next_coord not in lengths or new_length < lengths[next_coord]:
                    lengths[next_coord] = new_length
                    heapq.heappush(priority_queue, (new_length+ best_guess(next_coord, end) , new_length, next_coord))
    
    return -1

best  = find_route(maze, start, end)

for i in range(start_moving,len(obstacles)):
    maze.remove(obstacles[i])
    if find_route(maze, start,end) == -1:
        blockage = ','.join(map(str, obstacles[i]))
        break

print('Day 18 round 1 answer =', best)
print('Day 18 round 2 answer =', blockage)



Day 18 round 1 answer = 296
Day 18 round 2 answer = 28,44


### Day 19

In [9]:
# This is a "breadth first search". These seem to be a classic thing
# Only good when you can stop at a success

def can_build_pattern(pattern, towels, return_all = False):

    queue = deque([0])  
    visited = set()  
    success_count = 0
    while queue:
        start = queue.popleft()
        if start in visited and not return_all:
            continue

        visited.add(start)
        
        # Check all substrings starting from `start`
        for end in range(start + 1, len(pattern) + 1):
            chunk = pattern[start:end]
            if chunk in towels:
                if end == len(pattern) and not return_all:  # Successfully reached the end
                    return True
                elif end == len(pattern) and return_all:
                    success_count += 1

                queue.append(end)
    
    return success_count if return_all else False

def round_1(patterns, towels):
    answer = 0
    for i, pattern in enumerate(patterns):
        if can_build_pattern(pattern, towels):
            answer += 1
    return answer



#https://en.wikipedia.org/wiki/Trie

#" Every child node shares a common prefix with its parent node, and the root node represents the empty string."
# The idea is that a Trie is a very efficient way to search through strings, and is used for autocomplete algorithms
# I am combine this idea with Dynamic Programming, which uses feed forward algorithms that take a state and feed it into the next one
# So here we will count the number of ways to build up to character n and add then look at the next level

class TrieNode:
    # Node of the trie. Each node is a character, and sits in the tree based of what can come before and after it
    # Two important traits, what can come after (the children) and if we're at the end of a branch
    def __init__(self):
        self.children = {}
        self.is_end = False

class Trie:
    # Build the trie!
    # The key thing is that the trie is allllllll the towels but put together

    # It will have a base to start with

    # Then it will have the ability to add words into the tree which is done by finding the an appropriate branch to merge with or create

    # Then it will have the ability to search what came before it - ie search prefixes.
    # This will allow us to find all the possible branches that can be used in building our pattern
    
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        # To add a word we start at the top and then follow the correct branch until we need a new branch
        
        node = self.root
        for letter in word:
            if letter not in node.children:
                node.children[letter] = TrieNode()
            node = node.children[letter]
        node.is_end = True
    
    def search_prefixes(self, pattern , start_position_of_substring):
        # The idea is that when we have a pattern we are going to pass it into the true
        # We then want to know all the places that all the substrings appear in the tree
        # We will record it by recording what length of the pattern has been matched
        node = self.root
        matches = []
        for i in range(start_position_of_substring,len(pattern)):
            letter = pattern[i] #Remember nodes are defined by characters
            if letter not in node.children: 
                # At first this checks the root, which is equivalent to checking there is at least one towel starting with the required letter for this substring
                # When we do get matches we move along the branches as we move through the substring
                # We will break if at any point we stop finding matches
                # We will need the last match to be at the end of the branch
                break
            node = node.children[letter]

            if node.is_end:
                matches.append(i+1)
        return matches
    


def count_ways_to_build(pattern, towels):
    # Attempting to learn dynamical programming
    # This is all about propogating results through a dynamic table
    # Our table is going to be a line per monotonically increasing substring of the pattern
    # For example, if the pattern was abc, we will have 4 rows: "", "a","ab","abc" 
    # Each row will store the number of ways we can build that string, and of course the question wants the value in the last row
    trie = Trie()
    for towel in towels:
        trie.insert(towel)
    
    n = len(pattern)
    
    table_of_counts = [0]*(n+1)
    table_of_counts[0] = 1 # 1 way to make substring length 0, this is our initial value that will propagate through the table

    for i in range(n):
        if table_of_counts[i] > 0: # We will only process if we can actually reach i
            matches = trie.search_prefixes(pattern, i)
            for end in matches:
                table_of_counts[end] += table_of_counts[i]
    
    return table_of_counts[n]

def round_2(patterns, towels):
    answer = 0
    for pattern in patterns:
        ways = count_ways_to_build(pattern, towels)
        answer += ways
    return answer



towels, patterns = read_input(day=19, test=False, input_delimter='\n\n')
towels = set(towels.split(', '))
patterns = patterns.split('\n')


print('Day 19 round 1 answer =', round_1(patterns, towels))
print('Day 19 round 2 answer =', round_2(patterns, towels))



Day 19 round 1 answer = 333
Day 19 round 2 answer = 678536865274732


### Day 20 

In [9]:

all_objects = [(object, i, j) for i , line in enumerate( read_input(day=20, test=False)) for j, object in enumerate(line) ]
maze = {(i,j) for object, i , j in all_objects if object != '#'}
obstacles = {(i,j) for object, i , j in all_objects if object == '#'}

start , end = [(i,j) for object, i , j in all_objects if object == 'S'][0] ,[(i,j) for object, i , j in all_objects if object == 'E'][0]

def find_route(maze, start, end, cheat = None):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  
    if cheat:
        maze = maze | {cheat}

    def best_guess(current, goal):
        dx, dy = abs(current[0] - goal[0]), abs(current[1] - goal[1])
        return dx + dy

    priority_queue = [(0 + best_guess(start, end), 0, start)]  
    lengths = {start: 0}

    while priority_queue:
        _,length, current = heapq.heappop(priority_queue)

        if current == end:
            return length  

        for dx, dy in directions:
            next_coord = (current[0] + dx, current[1] + dy)
            if next_coord in maze:
                new_length = length + 1

                if next_coord not in lengths or new_length < lengths[next_coord]:
                    lengths[next_coord] = new_length
                    heapq.heappush(priority_queue, (new_length+ best_guess(next_coord, end) , new_length, next_coord))
    
    return -1


def find_all_paths(maze, start, end):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  
    best = float('inf')
    paths = []

        
    priority_queue = [(0, start, [start])]
    costs = defaultdict(lambda: float('inf')) 

    while priority_queue:

        cost, current, path = heapq.heappop(priority_queue)
        
        if cost > costs[current]:
            continue
        else:
            costs[current] = cost
        # End reached
        if current == end and cost <= best:
            best = cost
            paths.append(path)
            

        for new_direction, (dx, dy) in enumerate(directions):
            next_coord = (current[0] + dx, current[1] + dy)
            if next_coord in maze: 
                heapq.heappush(priority_queue, (cost+1, next_coord, path + [next_coord]))

    return best, paths

def find_cheat_candidates(maze, obstacles):
    def is_adjacent(coord1, coord2):
        return abs(coord1[0] - coord2[0]) + abs(coord1[1] - coord2[1]) == 1

    cheats = []
    for obstacle in obstacles:
        if obstacle[0] == 0 or obstacle[1] == 0:
            continue
        count = sum(is_adjacent(obstacle, maze_coord) for maze_coord in maze)
        if count >= 2:  # Check if adjacent to at least two coordinates in maze
            cheats.append(obstacle)
    return cheats

best = find_route(maze, start, end)
cheats = find_cheat_candidates(maze, obstacles)
saved = defaultdict(int)
for cheat in cheats:
    new_best = find_route(maze, start, end, cheat = cheat)
    if new_best < best:
        saved[best -new_best] +=1

print('Day 20 round 1 answer =', sum([ways for saving, ways in saved.items() if saving >= 100]))

best, paths  = find_all_paths(maze, start, end)

new_saved = defaultdict(int)
for path in paths:
    shortcuts = list(combinations(path, 2))
    for i, (shortcut_start, shortcut_end) in enumerate(shortcuts):
        dist = abs(shortcut_start[0] - shortcut_end[0]) + abs(shortcut_start[1] - shortcut_end[1])
        if dist <= 20:
            new_path = path[:path.index(shortcut_start)+1] + path[path.index(shortcut_end):]
            new_best = len(new_path) + dist -2
            saving = best - new_best
            if saving > 0:
                new_saved[saving] +=1 

print('Day 20 round 2 answer =', sum([ ways for saving, ways in new_saved.items() if saving >= 100]))



Day 20 round 1 answer = 1358
Day 20 round 2 answer = 1005856
